# Skript zum überprüfen logischer Fehler
## Vor einem Import in iDAI.chronontology

*Entwickelt von Lukas Lammers (DCH, Universität zu Köln) im Projekt FAIR.rdm, teil des DFG-geförderten SPP 2143 "Entangled Africa" - Version 1.0*

### Hintergrund

Dieses Skript untersützt beim Auffinden von Fehlern in Importtabellen in [iDAI.chronontology](https://chronontology.dainst.org/). Es ist Teil des von FAIR.rdm weiterentwickelten Import-Workflows (Schirtt IV). Fehler, die sonst nur während des Imports angezeigt werden würden, sollen hiermit frühzeitig identifiziert und behoben werden können. Außerdem überprüft es die logischen Beziehungen der Perioden untereinander, die während des Imports nicht getestet werden aber bei der Anzeige der Daten im Frontend zu erheblichen Schwierigkeiten führen können.

### Vorraussetzungen

Benötigt wird eine Python-Umgebung sowie die Bibliotheken pandas, requests und json, sowie eine Software zum Ausführend der Codezellen (wie bspw. VS Code).

### Ausführung

Die Zellen dieses Notebooks können der Reihe nach ausgeführt werden. Lediglich die Parameter der ersten Zelle müssen manuell angepasst werden. Wenn eine Zelle "fehlerfrei" ausgibt, kann mit der nächsten fortgefahren werden. Werden Fehler gefunden, folgen entsprechende Meldungen.

Wenn alle Zellen "fehlerfrei" melden, sind die Daten bereit für das Preprocessing.

### Resultat

Das Ergebis ist eine Logdatei aller Fehler, die im Ordner `/csv` gespeichert wird. Sie wird bei jedem Durchlauf überschrieben und hilft dabei, die Fehler der Reihe nach zu bearbeiten.




#### 1. Initialisierung

Hier muss `df_importtable`, `today` und `dataname` angepasst werden.

In [ ]:
#Import modules
import pandas as pd
import requests
import json

#Dieses Skript muss im Ordner "chronontology-data" liegen.

#Hier bitte Dateiname der Import-Tabelle angeben:
df_importtable = pd.read_csv('csv/2025/ChronOntology_ReImport_Bestandsabgleich_Korrektur_import_04-11-25.csv')
#Hier bitte das aktuelle Datum angeben:
today = '2025-11-19'
#Hier bitte den Namen der Fuhre angeben, er befindet sich im Anschluss im Dateinamen des Errorlogs:
dataname = 'Logic_Test_Test'

df_importtable = df_importtable.fillna('nd')

errorlog = []

print(df_importtable)

Dieser Test, ob die Daten korrekt eingelesen wurden, ist optional.

In [ ]:
#Einlestest
first_row = df_importtable.iloc[2]
print(first_row)

#### 2. Allgemeine ChronoID-Überprüfung

ChronoIDs werden durch den Import vergeben und müssen in der Konkordanzdatei vorhanden sein. Dies kann nur auf Perioden zutreffen, die bereits im System angelegt wurden und mit dem aktuellen Import geändert werden.

In [ ]:
df_konkordanz = pd.read_csv('csv/Konkordanz.csv')
#print(df_konkordanz)
all_chrono_ids = df_konkordanz['chronontologyID'].tolist()
print(all_chrono_ids)

def check_chronoid(row):
    if row['ChronoID'] != 'nd':
        if row['ChronoID'] in all_chrono_ids:
            error = "fehlerfrei"
            return error
        else:
            error = f"{row['importID']}: ChronoID ist unbekannt."
            return error
    else:
        pass

for index, row in df_importtable.iterrows():
    if row['ChronoID'] != 'nd':
        error = check_chronoid(row)
        if error != "fehlerfrei":
            errorlog.append(error)

print(check_chronoid(first_row))
print(errorlog)

#### 3. Allgemeine ImportID-Überprüfung. 

ImportIDs müssen eindeutig sein.

In [ ]:
import_id_list = df_importtable['importID'].tolist()
print(import_id_list)

#Kombination von ImportIDs und ChronoIDs in einer Liste für späteren Abgleich
all_ids_list = import_id_list + all_chrono_ids

def check_import_id(list_of_ids):
    import_id_set = set(import_id_list)
    if len(import_id_set) == len(list_of_ids):
        error = "fehlerfrei"
        return error
    else:
        error = "Fehler: ImportIDs mehrfach vergeben."
        return error

errorlog.append(check_import_id(import_id_list))
print(errorlog)

#### 4. Überprüfung der Names

Jede Zelle muss ein @ enthalten.

In [ ]:
def check_names(row):
    if '@' in row['names']:
        return "fehlerfrei"
    else:
        error = f"{row['importID']}: Name enthält keinen Sprachverweis."
        return error

for index, row in df_importtable.iterrows():
    error = check_names(row)
    if error != "fehlerfrei":
        errorlog.append(error)

print(check_names(first_row))
print(errorlog)

#### 5. Überprüfung der types

Die Zellen dürfen keine Leerzeichen enthalten.

In [ ]:
main_types = ['not_specified','geological','political','cultural','material_culture']
subtypes =['age','eon','epoch','era','period','supereon','architecture','art','ceramic','decorative_arts','fine_arts','glass','glass_vessels','glyptic','lacquer','metal_bowls','metal_ware','painting','photography','porcelain','pottery','printmaking','rock_art','sculpture','stone_tools','textile','wall_painting']

def check_types(row):
    #Überprüfe Leerzeichen
    if ' ' in row['types']:
        error = f"{row['importID']}: Type enthält Leerzeichen."
        return error
    else:
        #Überprüfe Typenzuordnung
        if ":" in row['types']:
            maintype = row['types'].split(':')[0]
            subtype = row['types'].split(':')[1]
            if maintype not in main_types:
                error = f"{row['importID']}: Haupt-Type {maintype} ist ungültig."
                return error
            elif subtype not in subtypes:
                error = f"{row['importID']}: Sub-Type {subtype} ist ungültig."
                return error
            else:
                error = "fehlerfrei"
                return error
        else:
            maintype = row['types']
            if maintype not in main_types:
                error = f"{row['importID']}: Haupt-Type {maintype} ist ungültig."
                return error
            else:
                error = "fehlerfrei"
                return error

for index, row in df_importtable.iterrows():
    error = check_types(row)
    if error != "fehlerfrei":
        errorlog.append(error)

print(check_types(first_row))
print(errorlog)

#### 6. Überprüfung der Gazetteer-IDs

Der folgende Code überprüft, ob unter den angegeben Gazetteer-Ids Datensätze im iDAI.gazetteer existieren. Da hierbei die API des iDAI.gazetteer abgefragt wird, kann der Vorgang etwas dauern.

In [ ]:
#testid = "2353208"

def check_gazetteer_id(gazetteer_id):
    url = f"https://gazetteer.dainst.org/place/{gazetteer_id}"
    response = requests.get(url)
    if response.status_code == 200:
        error = "fehlerfrei"
        return error
    else:
        error = f"{gazetteer_id}: ID im Gazetteer nicht gefunden."
        return error

for index, row in df_importtable.iterrows():
    if row['spatiallyPartOfRegion'] != 'nd':
        entry = str(row['spatiallyPartOfRegion'])
        if "," in entry:
            ids = entry.split(", ")
            for id in ids:
                if " " in id:
                    error = f"{row['importID']}: spatiallyPartOfRegion Leerzeichen in Gaz.-ID {id}"
                    errorlog.append(error)
                else:
                    errorlog.append(check_gazetteer_id(id))
        else:
            errorlog.append(check_gazetteer_id(entry))

for index, row in df_importtable.iterrows():
    if row['hasCoreArea'] != 'nd':
        entry = str(row['hasCoreArea'])
        if "," in entry:
            ids = entry.split(", ")
            for id in ids:
                if " " in id:
                    error = f"{row['importID']}: hasCoreArea Leerzeichen in Gaz.-ID {id}"
                    errorlog.append(error)
                else:
                    errorlog.append(check_gazetteer_id(id))
        else:
            errorlog.append(check_gazetteer_id(entry))

for index, row in df_importtable.iterrows():
    if row['isNamedAfter'] != 'nd':
        entry = str(row['isNamedAfter'])
        if "," in entry:
            ids = entry.split(", ")
            for id in ids:
                if " " in id:
                    error = f"{row['importID']}: isNamedAfter Leerzeichen in Gaz.-ID {id}"
                    errorlog.append(error)
                else:
                    errorlog.append(check_gazetteer_id(id))
        else:
            errorlog.append(check_gazetteer_id(entry))

#print(check_gazetteer_id(testid))

errorlog = set(errorlog)
errorlog = list(errorlog)

print(errorlog)

#### 7. Überprüfung von Schreibfehlern in den Relations

Die nächste Zelle sucht nach verstreuten Leerzeichen und Großbuchstaben, die durch die Autoformatierung in Tabellenkalkulationssoftware häufig auftreten.

In [ ]:
#Check for white spaces and uppercase in ID lists
def check_whitespace_and_case(df_importtable, column_name):
    errors = []
    
    for index, row in df_importtable.iterrows():
        cell_value = row[column_name]
        
        if pd.isna(cell_value) or cell_value == 'nd':
            continue
            
        cell_str = str(cell_value)
        
        # Zähle Kommas und Leerzeichen
        num_commas = cell_str.count(',')
        num_spaces = cell_str.count(' ')
        
        # Bei korrekter Formatierung: Anzahl Leerzeichen = Anzahl Kommas
        # (weil ", " das Trennzeichen ist)
        if num_spaces != num_commas:
            import_id = row['importID']
            errors.append(f"Fehler: Falsche Anzahl Leerzeichen in {column_name} bei {import_id} ('{cell_str}').")
        
        # Prüfe auf Großbuchstaben in IDs mit ':'
        # Splitte bei Komma und trimme
        ids = [id.strip() for id in cell_str.split(',')]
        for id in ids:
            if ':' in id:
                id_lower = id.lower()
                if id != id_lower:
                    import_id = row['importID']
                    errors.append(f"Fehler: Großbuchstabe in ID '{id}' bei {import_id}.")
    
    return errors if errors else "fehlerfrei"

# Check all columns and collect errors
result = check_whitespace_and_case(df_importtable, 'isPartOf')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'hasPart')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'isListedIn')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'lists')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'fallsWithin')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'contains')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'isFollowedBy')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'follows')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'isSenseOf')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'hasSense')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'sameAs')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'isSimilarTo')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

result = check_whitespace_and_case(df_importtable, 'differsInDefinitionFrom')
if isinstance(result, list):
    errorlog.extend(result)
elif result != "fehlerfrei":
    errorlog.append(result)

print(errorlog)

#### 8. Relationstest I

Diese Funktionen überprüfen Mehrfachbeziehungen zu gleichen Perioden.

In [ ]:
def split_ids(relation_string):
    if "," in relation_string:
        ids = relation_string.split(", ")
        return ids
    else:
        ids = [relation_string]
        return ids

def unique_relation(row):
    relation_list = []
    if row['isPartOf'] != 'nd':
        relation_list.extend(split_ids(row['isPartOf']))
    if row['hasPart'] != 'nd':
        relation_list.extend(split_ids(row['hasPart']))
    if row['isListedIn'] != 'nd':
        relation_list.extend(split_ids(row['isListedIn']))
    if row['lists'] != 'nd':
        relation_list.extend(split_ids(row['lists']))
    if row['fallsWithin'] != 'nd':
        relation_list.extend(split_ids(row['fallsWithin']))
    if row['contains'] != 'nd':
        relation_list.extend(split_ids(row['contains']))
    if row['isFollowedBy'] != 'nd':
        relation_list.extend(split_ids(row['isFollowedBy']))
    if row['follows'] != 'nd':
        relation_list.extend(split_ids(row['follows']))
    if row['isSenseOf'] != 'nd':
        relation_list.extend(split_ids(row['isSenseOf']))
    if row['hasSense'] != 'nd':
        relation_list.extend(split_ids(row['hasSense']))
    if row['sameAs'] != 'nd':
        relation_list.extend(split_ids(row['sameAs']))
    if row['isSimilarTo'] != 'nd':
        relation_list.extend(split_ids(row['isSimilarTo']))
    if row['differsInDefinitionFrom'] != 'nd':
        relation_list.extend(split_ids(row['differsInDefinitionFrom']))
    
    relation_set = set(relation_list)
    if len(relation_set) == len(relation_list):
        error = "fehlerfrei"
        return error
    else:
        error = f"{row['importID']}: Mehrfachbeziehungen zu gleichen Perioden."
        return error
    
print(unique_relation(first_row))

for index, row in df_importtable.iterrows():
    error = unique_relation(row)
    if error != "fehlerfrei":
        errorlog.append(error)

print(errorlog)

#### 9. Relationstest II

Die nächsten drei Zellen bereiten Funktonionen vor, mit denen die einzlenen Relationstypen überprüft werden.

In [ ]:
#Diese Funktion konvertiert die Zelleninhalte in Listen, um sie später überprüfen zu können.
def cell_to_list(row, column_name):
    if row[column_name] != 'nd':
        data = row[column_name].replace(' ', '')
        return data.split(',')
    else:
        return []
    
print(cell_to_list(first_row, 'fallsWithin'))

In [ ]:
#Diese Funktion extrahiert Start- und Enddaten einer Periode aus der Tabelle.
print(first_row['timeStandardized'])

def extract_startdate_from_table(row):
    date = row['timeStandardized']
    date = date.replace(' ', '')
    date = date.replace('[', '')
    date = date.replace(']', '')
    if date[0] == ';':
        startdate = None
    else:
        date = date.split(';')[0]
        if "/" in date:
            date = date.split('/')
            a = int(date[0])
            b = int(date[1])
            if a < b:
                startdate = a
            else:
                startdate = b
        else:
            startdate = int(date)
    return startdate

def extract_enddate_from_table(row):
    date = row['timeStandardized']
    date = date.replace(' ', '')
    date = date.replace('[', '')
    date = date.replace(']', '')
    if date[-1] == ';':
        enddate = None
    else:
        date = date.split(';')[1]
        if "/" in date:
            date = date.split('/')
            a = int(date[0])
            b = int(date[1])
            if a > b:
                enddate = a
            else:
                enddate = b
        else:
            enddate = int(date)
    return enddate

print(extract_startdate_from_table(first_row))
print(extract_enddate_from_table(first_row))

In [ ]:
#Diese Funktionen extrahieren Start- und Enddaten einer Periode über die API.

def extract_startdate_from_api(chronoID):
    url = f"https://chronontology.dainst.org/data/period/{chronoID}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        #Check for timespan
        if 'hasTimespan' in data['resource']:
            timespan = data['resource']['hasTimespan'][0]
            if 'begin' in timespan:
                startdate = timespan['begin']
                if 'at' in startdate:
                    startdate = startdate['at']
                    if startdate == 'not specified':
                        startdate = None
                    else:
                        startdate = int(startdate)
                elif 'notBefore' in startdate:
                    startdate = startdate['notBefore']
                    if startdate == 'not specified':
                        startdate = None
                    else:
                        startdate = int(startdate)
                else:
                    startdate = None
            else:
                startdate = None
        else:
            startdate = None
    else:
        startdate = None
    return startdate

def extract_enddate_from_api(chronoID):
    url = f"https://chronontology.dainst.org/data/period/{chronoID}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'hasTimespan' in data['resource']:
            timespan = data['resource']['hasTimespan'][0]
            if 'end' in timespan:
                enddate = timespan['end']
                if 'at' in enddate:
                    enddate = enddate['at']
                    if enddate == 'not specified':
                        enddate = None
                    else:
                        enddate = int(enddate)
                elif 'notAfter' in enddate:
                    enddate = enddate['notAfter']
                    if enddate == 'not specified':
                        enddate = None
                    else:
                        enddate = int(enddate)
                else:
                    enddate = None
            else:
                enddate = None
        else:
            enddate = None
    else:
        enddate = None
    return enddate

print(extract_startdate_from_api(first_row))
print(extract_enddate_from_api(first_row))
         

In [ ]:
def check_ispartof(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['isPartOf'] != 'nd':
        is_part_of_list = cell_to_list(row, 'isPartOf')
        for id in is_part_of_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: isPartOf Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period > enddate_matching:
                            error = f"{row['importID']}: isPartOf Endzeit übertrifft die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: isPartOf enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            other_ids.append(matching_row['follows'].tolist())
            #Check if id is in incorrect relations
            if id in other_ids:
                error = f"{row['importID']}: isPartOf enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            #If not, check if dates are logical
            elif id in matching_row['hasPart'].tolist():
                startdate_period = extract_startdate_from_table(row)
                enddate_period = extract_enddate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                enddate_matching = extract_enddate_from_table(matching_row.iloc[0])
                if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                    if startdate_period < startdate_matching:
                        error = f"{row['importID']}: isPartOf Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                        return error
                    elif enddate_period > enddate_matching:
                        error = f"{row['importID']}: isPartOf Endzeit übertrifft die von {id}."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: isPartOf Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period > enddate_matching:
                            error = f"{row['importID']}: isPartOf Endzeit übertrifft die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: isPartOf enthält die unbekannte ID: {id}."
                    return error
    else:
        error = "fehlerfrei"
        return error
    
for index, row in df_importtable.iterrows():
    error = check_ispartof(row)
    if error != "fehlerfrei":
        errorlog.append(error)


print(errorlog)


In [ ]:
def check_haspart(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['hasPart'] != 'nd':
        is_part_of_list = cell_to_list(row, 'hasPart')
        for id in is_part_of_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period > startdate_matching:
                            error = f"{row['importID']}: hasPart Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period < enddate_matching:
                            error = f"{row['importID']}: hasPart Endzeit unterschreitet die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: hasPart enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            other_ids.append(matching_row['follows'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: hasPart enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['isPartOf'].tolist():
                #error = "fehlerfrei"
                startdate_period = extract_startdate_from_table(row)
                enddate_period = extract_enddate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                enddate_matching = extract_enddate_from_table(matching_row.iloc[0])
                if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                    if startdate_period > startdate_matching:
                        error = f"{row['importID']}: hasPart Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                        return error
                    elif enddate_period < enddate_matching:
                        error = f"{row['importID']}: hasPart Endzeit unterschreitet die von {id}."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period > startdate_matching:
                            error = f"{row['importID']}: hasPart Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period < enddate_matching:
                            error = f"{row['importID']}: hasPart Endzeit unterschreitet die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: hasPart enthält die unbekannte ID: {id}."
                    return error
    else:
        return "fehlerfrei"


In [ ]:
def check_islistedin(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['isListedIn'] != 'nd':
        is_listed_in_list = cell_to_list(row, 'isListedIn')
        for id in is_listed_in_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: isListedIn enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            other_ids.append(matching_row['follows'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: isListedIn enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['lists'].tolist():
                error = "fehlerfrei"
                return error
            else:
                if id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                elif current_chrono_id not in cell_to_list(matching_row.iloc[0], 'lists') and current_import_id not in cell_to_list(matching_row.iloc[0], 'lists'):
                    error = f"{row['importID']}: Warnung: isListedIn/lists in {id} nicht rekursiv eingetragen."
                    return error
                else:
                    error = "fehlerfrei"
                    return error
    else:
        return "fehlerfrei"


In [ ]:
def check_lists(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['lists'] != 'nd':
        lists_list = cell_to_list(row, 'lists')
        for id in lists_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: lists enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            other_ids.append(matching_row['follows'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: lists enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['isListedIn'].tolist():
                error = "fehlerfrei"
                return error
            else:
                if id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                elif current_chrono_id not in cell_to_list(matching_row.iloc[0], 'isListedIn') and current_import_id not in cell_to_list(matching_row.iloc[0], 'isListedIn'):
                    error = f"{row['importID']}: Warnung: isListedIn/lists in {id} nicht rekursiv eingetragen."
                    return error
                else:
                    error = "fehlerfrei"
                    return error
    else:
        return "fehlerfrei"


In [ ]:
def check_fallswithin(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['fallsWithin'] != 'nd':
        falls_within_list = cell_to_list(row, 'fallsWithin')
        for id in falls_within_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: fallsWithin Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period > enddate_matching:
                            error = f"{row['importID']}: fallsWithin Endzeit übertrifft die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: fallsWithin enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            other_ids.append(matching_row['follows'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: fallsWithin enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['contains'].tolist():
                #error = "fehlerfrei"
                startdate_period = extract_startdate_from_table(row)
                enddate_period = extract_enddate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                enddate_matching = extract_enddate_from_table(matching_row.iloc[0])
                if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                    if startdate_period < startdate_matching:
                        error = f"{row['importID']}: fallsWithin Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                        return error
                    elif enddate_period > enddate_matching:
                        error = f"{row['importID']}: fallsWithin Endzeit übertrifft die von {id}."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: fallsWithin Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period > enddate_matching:
                            error = f"{row['importID']}: fallsWithin Endzeit übertrifft die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: fallsWithin enthält die unbekannte ID: {id}."
                    return error
    else:
        return "fehlerfrei"


In [ ]:
def check_contains(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['contains'] != 'nd':
        contains_list = cell_to_list(row, 'contains')
        for id in contains_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period > startdate_matching:
                            error = f"{row['importID']}: contains Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period < enddate_matching:
                            error = f"{row['importID']}: contains Endzeit unterschreitet die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: contains enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['isFollowedBy'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: contains enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['fallsWithin'].tolist():
                #error = "fehlerfrei"
                startdate_period = extract_startdate_from_table(row)
                enddate_period = extract_enddate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                enddate_matching = extract_enddate_from_table(matching_row.iloc[0])
                if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                    if startdate_period > startdate_matching:
                        error = f"{row['importID']}: contains Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                        return error
                    elif enddate_period < enddate_matching:
                        error = f"{row['importID']}: contains Endzeit unterschreitet die von {id}."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    enddate_period = extract_enddate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    enddate_matching = extract_enddate_from_api(id)
                    if startdate_period and enddate_period and startdate_matching and enddate_matching != None:
                        if startdate_period > startdate_matching:
                            error = f"{row['importID']}: contains Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                            return error
                        elif enddate_period < enddate_matching:
                            error = f"{row['importID']}: contains Endzeit unterschreitet die von {id}."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: contains enthält die unbekannte ID: {id}."
                    return error
    else:
        return "fehlerfrei"


In [ ]:
def check_isfollowedby(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['isFollowedBy'] != 'nd':
        is_followed_by_list = cell_to_list(row, 'isFollowedBy')
        for id in is_followed_by_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    if startdate_period and startdate_matching != None:
                        if startdate_period > startdate_matching:
                            error = f"{row['importID']}: isFollowedBy Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: isFollowedBy enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: isFollowedBy enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['follows'].tolist():
                #error = "fehlerfrei"
                startdate_period = extract_startdate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                if startdate_period and startdate_matching != None:
                    if startdate_period > startdate_matching:
                        error = f"{row['importID']}: isFollowedBy Anfangszeit ({startdate_period}) nach der von {id} ({startdate_matching})."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if current_chrono_id not in cell_to_list(matching_row.iloc[0], 'follows') and current_import_id not in cell_to_list(matching_row.iloc[0], 'follows'):
                    error = f"{row['importID']}: Warnung: isFollowedBy/follows in {id} nicht rekursiv eingetragen."
                    return error
                else:
                    error = "fehlerfrei"
                    return error
    else:
        return "fehlerfrei"

print(check_isfollowedby(first_row))

In [ ]:
def check_follows(row):
    current_chrono_id = row['ChronoID']
    current_import_id = row['importID']
    #Check if relation is defined
    if row['follows'] != 'nd':
        follows_list = cell_to_list(row, 'follows')
        for id in follows_list:
            #get the row where id is present in ChronoID oder importID
            matching_row = df_importtable[(df_importtable['ChronoID'] == id) | (df_importtable['importID'] == id)]
            
            # Check if matching_row is empty
            if matching_row.empty:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    if startdate_period and startdate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: follows Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif id in all_ids_list:
                    error = "fehlerfrei"
                    return error
                else:
                    error = f"{row['importID']}: follows enthält die unbekannte ID: {id}."
                    return error
            
            #print(matching_row)
            other_ids = []
            other_ids.append(matching_row['isPartOf'].tolist())
            other_ids.append(matching_row['hasPart'].tolist())
            other_ids.append(matching_row['isListedIn'].tolist())
            other_ids.append(matching_row['lists'].tolist())
            other_ids.append(matching_row['fallsWithin'].tolist())
            other_ids.append(matching_row['contains'].tolist())
            if id in other_ids:
                error = f"{row['importID']}: follows enthält eine ID, die in {matching_row['importID']} anders definiert ist."
                return error
            elif id in matching_row['isFollowedBy'].tolist():
                #error = "fehlerfrei"
                startdate_period = extract_startdate_from_table(row)
                startdate_matching = extract_startdate_from_table(matching_row.iloc[0])
                if startdate_period and startdate_matching != None:
                    if startdate_period < startdate_matching:
                        error = f"{row['importID']}: follows Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                        return error
                    else:
                        error = "fehlerfrei"
                        return error
                else:
                    error = "fehlerfrei"
                    return error
            else:
                if id in all_chrono_ids:
                    #error = "fehlerfrei"
                    startdate_period = extract_startdate_from_api(row)
                    startdate_matching = extract_startdate_from_api(id)
                    if startdate_period and startdate_matching != None:
                        if startdate_period < startdate_matching:
                            error = f"{row['importID']}: follows Anfangszeit ({startdate_period}) vor der von {id} ({startdate_matching})."
                            return error
                        else:
                            error = "fehlerfrei"
                            return error
                    else:
                        error = "fehlerfrei"
                        return error
                elif current_chrono_id not in cell_to_list(matching_row.iloc[0], 'isFollowedBy') and current_import_id not in cell_to_list(matching_row.iloc[0], 'isFollowedBy'):
                    error = f"{row['importID']}: Warnung: isFollowedBy/follows in {id} nicht rekursiv eingetragen."
                    return error
                else:
                    if id in all_ids_list:
                        error = "fehlerfrei"
                        return error
                    else:
                        error = f"{row['importID']}: follows enthält die unbekannte ID: {id}."
                        return error
    else:
        return "fehlerfrei"
    
print(check_follows(first_row))

In [ ]:
#Loop zum Überprüfen der Beziehungen

x = len(df_importtable)

for index, row in df_importtable.iterrows():
    print(f"Überprüfe Zeile {index + 1} von {x}...")
    error = check_ispartof(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_haspart(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_islistedin(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_lists(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_fallswithin(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_contains(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_isfollowedby(row)
    if error != "fehlerfrei":
        errorlog.append(error)
    error = check_follows(row)
    if error != "fehlerfrei":
        errorlog.append(error)

#### 10. Abschluss des Tests

Gibt die letzte Codezelle "Alle Beziehungen sind fehlerfrei." aus, sind die Daten wahrscheinilch in Ordnung und das Preprocessing kann beginnen.

Wenn Fehler gefunden wurden, wird eine Liste im `csv`-Ordner erzeugt. Damit können die einzelnen Fehler der Reihe nach korrigiert werden. Es empfiehlt sich, dieses Skript danach erneut auszuführen.

In [ ]:
#print(len(errorlog))

errorlog = [e for e in errorlog if e not in ("fehlerfrei", None, "")]

#print(len(errorlog))
print(errorlog)

if len(errorlog) == 0:
    print("Alle Beziehungen sind fehlerfrei.")
else:
    #Save Errorlof to csv with the columns 'error' and an empty 'comment' column
    df_errorlog = pd.DataFrame(errorlog, columns=['error'])
    df_errorlog['comment'] = ''
    df_errorlog.to_csv(f'csv/errorlog_{dataname}_{today}.csv', index=False)
    print("Fehler wurden im Errorlog gespeichert. Bitte Fehler beheben und dieses Skript im Anschluss erneut ausführen.")